In [5]:
from pprint import pprint
import sys
import os
import numpy as np
import random

gpu = 7

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
print('os.environ["CUDA_VISIBLE_DEVICES"] : ',os.environ["CUDA_VISIBLE_DEVICES"])
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()
print('tf.executing_eagerly()', tf.executing_eagerly())

from stuf import stuf
from pyleaves.configs.config_v2 import BaseConfig
import json

sys.argv.extend(['--num_epochs','10'])

try:
    config = BaseConfig().parse(args=sys.argv[1:])
except Exception as e:
    print(e)
    print("missing or invalid arguments")
    print('sys.argv = ', sys.argv[1:])
    exit(0)

from pyleaves.leavesdb.tf_utils.tf_utils import set_random_seed
set_random_seed(config.seed)

from pyleaves.base.base_data_manager import DataManager
from pyleaves.base.base_trainer import ModelBuilder, BaseTrainer
from pyleaves.train.callbacks import get_callbacks
from pyleaves.loggers.mlflow_logger import MLFlowLogger as Logger

data_manager = DataManager(config=config)
train_data = data_manager.get_data_loader(file_group='train')
val_data = data_manager.get_data_loader(file_group='val')
test_data = data_manager.get_data_loader(file_group='test')
# import pdb; pdb.set_trace()

model_builder = ModelBuilder(config)
callbacks = get_callbacks(weights_best=os.path.join(config.model_config.model_dir,'weights_best.h5'),
                              logs_dir=os.path.join(config.model_config.log_dir,'tensorboard_logs'),
                              val_data=val_data,
                              batches_per_epoch=30,
                              freq=5,
                              histogram_freq=0,
                              restore_best_weights=True,
                              seed=2039)


logger = Logger(config)

trainer = BaseTrainer(config, model_builder, data_manager, logger, callbacks)






# from pyleaves.models import vgg16 

# vgg16.get_vgg16_conv_block_outputs(trainer.model.layers[0])

trainer.train()
trainer.save_model(config.run_id+'_model')
model = trainer.load_model(config.run_id+'_model')
# os.listdir(r'/media/data_cifs/jacob/Fossil_Project/models/PNAS/vgg16_grayscale-saved_base_model/variables')
trainer.test()



In [ ]:
# trainer.model_builder.model == 
trainer.model.weights = model.weights

np.all(model.weights==trainer.model_builder.model.weights)



# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
with tf.Graph().as_default():
    base = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                            include_top=False)

help(tf.keras.models.load_model)

tf.keras.__version__



